In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.sparse import csr_matrix
import sklearn
import itertools
import scipy as sp
from scipy import spatial
from sklearn.metrics.pairwise import cosine_similarity
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [2]:
data = pd.read_csv("movie_recommendation.csv")
movies = pd.read_csv("movies2.csv")

In [3]:
data

,userId,movieId,rating,date,title,genres,link,published year
0,1.0,1.0,4.0,30-07-00,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,www.imdb.com/title/tt114709,(1995)
1,5.0,1.0,4.0,08-11-96,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,www.imdb.com/title/tt114709,(1995)
2,7.0,1.0,4.5,25-01-05,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,www.imdb.com/title/tt114709,(1995)
3,15.0,1.0,2.5,13-11-17,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,www.imdb.com/title/tt114709,(1995)
4,17.0,1.0,4.5,18-05-11,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,www.imdb.com/title/tt114709,(1995)
...,...,...,...,...,...,...,...,...
96644,610.0,160341.0,2.5,19-11-16,Bloodmoon (1997),Action|Thriller,www.imdb.com/title/tt118745,(1997)
96645,610.0,160527.0,4.5,19-11-16,Sympathy for the Underdog (1971),Action|Crime|Drama,www.imdb.com/title/tt66806,(1971)
96646,610.0,160836.0,3.0,03-05-17,Hazard (2005),Action|Drama|Thriller,www.imdb.com/title/tt798722,(2005)
96647,610.0,163937.0,3.5,04-05-17,Blair Witch (2016),Horror|Thriller,www.imdb.com/title/tt1540011,(2016)


## Naiev collaborative filltering

In [4]:
movie_user_mat = data.pivot(index='movieId', columns='userId', values='rating').fillna(0)

In [5]:
movie_user_mat

userId,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,601.0,602.0,603.0,604.0,605.0,606.0,607.0,608.0,609.0,610.0
movieId,,,,,,,,,,,,,,,,,,,,,
1.0,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3.0,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
movie_user_mat_sparse = csr_matrix(movie_user_mat.values)

In [7]:
movie_user_mat_sparse

<9472x610 sparse matrix of type '<class 'numpy.float64'>'
	with 96649 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.neighbors import NearestNeighbors
# define model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
# fit
model_knn.fit(movie_user_mat_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [9]:
df_movies = movies[["movieId","title"]]

In [10]:
movie_to_idx = {
    movie: i for i, movie in 
    enumerate(list(df_movies.set_index('movieId').loc[movie_user_mat.index].title))
}



In [11]:
from fuzzywuzzy import fuzz

def fuzzy_matching(mapper, fav_movie, verbose=True):
    """
    return the closest match via fuzzy ratio. 
    
    Parameters
    ----------    
    mapper: dict, map movie title name to index of the movie in data
    fav_movie: str, name of user input movie
    
    verbose: bool, print log if True
    Return
    ------
    index of the closest match
    """
    match_tuple = []
    # get match
    for title, idx in mapper.items():
        ratio = fuzz.ratio(title.lower(), fav_movie.lower())
        if ratio >= 60:
            match_tuple.append((title, idx, ratio))
    # sort
    match_tuple = sorted(match_tuple, key=lambda x: x[2])[::-1]
    if not match_tuple:
        print('Oops! No match is found')
        return
    if verbose:
        print('Found possible matches in our database: {0}\n'.format([x[0] for x in match_tuple]))
    return match_tuple[0][1]

/home/mohammad/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [12]:
def make_recommendation(model_knn, data, mapper, fav_movie, n_recommendations):
    """
    return top n similar movie recommendations based on user's input movie
    Parameters
    ----------
    model_knn: sklearn model, knn model
    data: movie-user matrix
    mapper: dict, map movie title name to index of the movie in data
    fav_movie: str, name of user input movie
    n_recommendations: int, top n recommendations
    Return
    ------
    list of top n similar movie recommendations
    """
    # fit
    model_knn.fit(data)
    
    # get input movie index
    idx = fuzzy_matching(mapper, fav_movie, verbose=True)
    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    
    raw_recommends = sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    # get reverse mapper
    reverse_mapper = {v: k for k, v in mapper.items()}
    # print recommendations
    print('Recommendations for {}:'.format(fav_movie))
    test_array = []
    for i, (idx, dist) in enumerate(raw_recommends):
        test_array.append(reverse_mapper[idx])
    
    return test_array

In [13]:
test_array = []

def test(my_favorite):
        test_array.append(make_recommendation(
        model_knn=model_knn,
        data=movie_user_mat_sparse,
        fav_movie=my_favorite,
        mapper=movie_to_idx,
        n_recommendations=10))
    
my_favorite = ["toy story"]
for movie in my_favorite:
    test(movie)

Found possible matches in our database: ['Toy Story (1995)', 'Toy Story 3 (2010)', 'Toy Story 2 (1999)']

Recommendations for toy story:


In [14]:
test_array

[['Back to the Future (1985)',
  'Groundhog Day (1993)',
  'Mission: Impossible (1996)',
  'Lion King, The (1994)',
  'Star Wars: Episode VI - Return of the Jedi (1983)',
  'Forrest Gump (1994)',
  'Star Wars: Episode IV - A New Hope (1977)',
  'Independence Day (a.k.a. ID4) (1996)',
  'Jurassic Park (1993)',
  'Toy Story 2 (1999)']]

## -----------------------------------------------------------------------------------------------

## ----------------------------------------------------------------------------------------------

In [15]:
data2 = data[["userId", "movieId", "rating"]]
group1 = data2.groupby(['userId'],as_index = False, sort=False)["rating"].sum()
data2 = pd.merge(data2, group1, on = "userId", how = "left", sort = False)
group2 = data2.groupby(['userId'],as_index = False, sort=False)["rating_x"].count()
data2 = pd.merge(data2, group2, on = "userId", how = "left", sort = False)
data2["normalize_rating"] = (data2["rating_x_x"] - data2["rating_y"]) / data2["rating_x_y"]
data_normalize = data2.rename(columns = {"rating_x_x":"rating", "rating_y":"sum", "rating_x_y":"count"})

In [16]:
user_similarity_mat = data_normalize.pivot(index='movieId', columns='userId', values='normalize_rating').fillna(0)
user_similarity_mat.reset_index(inplace=True)
user_similarity_mat = user_similarity_mat.drop('movieId', axis=1)
user_ratings = pd.DataFrame(user_similarity_mat, index = user_similarity_mat.columns, columns = user_similarity_mat.columns)
similarity = cosine_similarity(user_ratings)
user_ratings = pd.DataFrame(similarity, index = user_ratings.index, columns = user_ratings.index)

#piv_sparse = sp.sparse.csr_matrix(piv_norm.values)
#user_similarity = 1 - spatial.distance.cosine(piv_sparse)
#user_similarity = cosine_similarity(piv_sparse)


for i in range(len(user_ratings.columns)) :
    for j in range(len(user_ratings.columns)) :
        user_ratings.iloc[i,j] = 1-cosine(user_similarity_mat.iloc[:,i],user_similarity_mat.iloc[:,j])

In [17]:
user_ratings

userId,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,601.0,602.0,603.0,604.0,605.0,606.0,607.0,608.0,609.0,610.0
userId,,,,,,,,,,,,,,,,,,,,,
1.0,1.000000,0.296749,0.102113,0.293768,0.314169,0.217245,0.159211,0.052030,0.424438,0.321074,...,0.237530,0.179380,0.042590,0.205315,0.000000,0.099750,0.094053,0.293002,0.0,0.461428
2.0,0.296749,1.000000,0.056821,0.391532,0.291379,0.389498,0.234639,0.285807,0.230094,0.234991,...,0.259767,0.408073,0.063824,0.252912,0.055684,0.000000,0.140795,0.192905,0.0,0.371397
3.0,0.102113,0.056821,1.000000,0.165346,0.081455,0.266181,0.152381,0.000000,0.102834,0.140527,...,0.000000,0.000000,0.000000,0.089950,0.000000,0.000000,0.403571,0.031541,0.0,0.050519
4.0,0.293768,0.391532,0.165346,1.000000,0.327143,0.483155,0.234873,0.350648,0.242747,0.271483,...,0.133982,0.381538,0.000000,0.262868,0.000000,0.000000,0.140530,0.136777,0.0,0.323699
5.0,0.314169,0.291379,0.081455,0.327143,1.000000,0.276292,0.165698,0.215349,0.394779,0.285038,...,0.225470,0.289577,0.043590,0.144722,0.086182,0.102109,0.096293,0.431290,0.0,0.450092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606.0,0.099750,0.000000,0.000000,0.000000,0.102109,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.101535,0.0,0.073100
607.0,0.094053,0.140795,0.403571,0.140530,0.096293,0.138522,0.377581,0.000000,0.085749,0.117233,...,0.000000,0.000000,0.000000,0.222885,0.000000,0.000000,1.000000,0.000000,0.0,0.068873
608.0,0.293002,0.192905,0.031541,0.136777,0.431290,0.148513,0.097526,0.097012,0.254694,0.215975,...,0.106141,0.191832,0.125502,0.108406,0.085728,0.101535,0.000000,1.000000,0.0,0.378927


def sim_movies(users, user_movie):
    movies = []
    for user in users:
        movies.append(data[data["userId"] == user[0]]["title"])
    movies = list(itertools.chain.from_iterable(movies))
    movies = list(set(movies))
    movies = set(movies) - set(user_movie)
    return movies

In [18]:
def top_users(user):
    users = []
    #if user not in user_sim_df.columns:
      #  return('No data available on user {}'.format(user))
    
    sim_values = user_ratings.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:11]
    sim_users = user_ratings.sort_values(by=user, ascending=False).index[1:11]
    zipped = zip(sim_users, sim_values,)
    for user, sim in zipped:
        users.append(user)
    #movies = sim_movies(users, user_movie)
    return users

In [19]:
sim_user = top_users(1)
#sim_user.append(1.0)
sim_user_data = pd.DataFrame()
sim_user.append(1)

for user in sim_user:
    sim_user_data = sim_user_data.append(data[data["userId"] == user])

### -----------------------------------------------------------------------------------------------------------------------
### -----------------------------------------------------------------------------------------------------------------------
## movie based

In [20]:
movie_similarity_mat = sim_user_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)
df = movie_similarity_mat.replace(0, np.NaN)
df = df.mean() 
movie_similarity_mat = movie_similarity_mat.replace(0, np.NaN)
for column in movie_similarity_mat.columns:
    movie_similarity_mat[column] = movie_similarity_mat[column].fillna(df[column]) 

In [21]:
movie_similarity_mat

movieId,1.0,2.0,3.0,5.0,6.0,7.0,8.0,10.0,11.0,15.0,...,179819.0,180031.0,180045.0,180095.0,180497.0,180777.0,180985.0,184791.0,185135.0,187595.0
userId,,,,,,,,,,,,,,,,,,,,,
1.0,4.000000,3.75,4.000000,2.5,4.000000,3.0,3.0,3.0,5.0,2.0,...,4.5,4.0,4.0,3.0,4.0,4.5,3.5,2.5,4.5,3.5
18.0,3.500000,3.00,3.666667,2.5,4.000000,3.0,3.0,3.0,5.0,2.0,...,4.5,4.0,4.0,3.0,4.0,4.5,3.5,2.5,4.5,3.5
275.0,5.000000,3.75,3.666667,2.5,3.666667,3.0,3.0,3.0,5.0,2.0,...,4.5,4.0,4.0,3.0,4.0,4.5,3.5,2.5,4.5,3.5
321.0,3.833333,5.00,3.000000,3.0,3.666667,3.0,3.0,3.0,5.0,2.0,...,4.5,4.0,4.0,3.0,4.0,4.5,3.5,2.5,4.5,3.5
324.0,3.833333,3.75,3.666667,2.5,3.666667,3.0,3.0,3.0,5.0,2.0,...,4.5,4.0,4.0,3.0,4.0,4.5,3.5,2.5,4.5,3.5
414.0,4.000000,3.00,4.000000,2.0,3.000000,3.0,3.0,3.0,5.0,2.0,...,4.5,4.0,4.0,3.0,4.0,4.5,3.5,2.5,4.5,3.5
432.0,2.500000,4.00,3.666667,2.5,3.666667,3.0,3.0,3.0,5.0,2.0,...,4.5,4.0,4.0,3.0,4.0,4.5,3.5,2.5,4.5,3.5
479.0,3.833333,3.75,3.666667,2.5,3.666667,3.0,3.0,3.0,5.0,2.0,...,4.5,4.0,4.0,3.0,4.0,4.5,3.5,2.5,4.5,3.5
500.0,4.000000,3.75,3.666667,2.5,3.666667,3.0,3.0,3.0,5.0,2.0,...,4.5,4.0,4.0,3.0,4.0,4.5,3.5,2.5,4.5,3.5


In [22]:
recommended_movies = movie_similarity_mat.loc[1]
recommended_movies = pd.DataFrame(recommended_movies).reset_index()
recommended_movies = recommended_movies.rename(columns = {1:'ratings'})

In [23]:
watched_movies = data[data["userId"] == 1]["movieId"]
watched_movies = pd.DataFrame(watched_movies)

In [24]:
cond = recommended_movies['movieId'].isin(watched_movies['movieId'])
recommended_movies.drop(recommended_movies[cond].index, inplace = True)

In [25]:
recommended_movies #= recommended_movies.sort_values(by=['ratings'], ascending = False) 

,movieId,ratings
1,2.0,3.75
3,5.0,2.50
5,7.0,3.00
6,8.0,3.00
7,10.0,3.00
...,...,...
3037,180777.0,4.50
3038,180985.0,3.50
3039,184791.0,2.50
3040,185135.0,4.50


In [26]:
recommended_movies_title = []
for Id in recommended_movies["movieId"]:
    recommended_movies_title.append(df_movies[df_movies["movieId"] == Id]["title"])
recommended_movies_title = list(itertools.chain.from_iterable(recommended_movies_title))

In [27]:
recommended_movies_title = recommended_movies["ratings"][0:10]

for i in range(len(movie_ratings.columns)) :
    for j in range(len(movie_ratings.columns)) :
        movie_ratings.iloc[i,j] = 1-cosine(movie_similarity_mat.iloc[:,i],movie_similarity_mat.iloc[:,j])

y_pred = []
y_test = []

for movie in intersect_movies:
    y_pred.append(df[movie])
    y_test.append(watched_movies[watched_movies["movieId"] == movie]["rating"].tolist())

import math
import sklearn.metrics as metrics
print(np.sqrt(sklearn.metrics.mean_squared_error(y_test, y_pred)))

train_data = data["title"]
train_data = list(dict.fromkeys(train_data))

-----------------------------------------------------------------------------------------------

In [210]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [211]:
spark = SparkSession.builder.appName('Recommendation_system').getOrCreate()

df = spark.read.csv("movie_recommendation.csv" , inferSchema = True , header = True)
df_rating = df.select(df['userId'],df['movieId'],df['rating'])

(training, test) = df_rating.randomSplit([0.8, 0.2])

als_model = ALS(maxIter = 10,regParam = 0.01, rank = 10, userCol = "userId", itemCol = "movieId",ratingCol = "rating", coldStartStrategy = "drop")
model = als_model.fit(training)

predictions = model.transform(test)

spark.createDataFrame
prediction = predictions.select("*").toPandas()

user_recs = model.recommendForAllUsers(20)

spark.createDataFrame
result_recomendation_user = user_recs.select("*").toPandas()

In [212]:
result_recomendation_user

,userId,recommendations
0,471,"[(46965, 7.621029853820801), (102903, 7.244442..."
1,463,"[(102123, 7.234038829803467), (54256, 7.084131..."
2,496,"[(3468, 6.239795684814453), (1212, 6.114135265..."
3,148,"[(3444, 6.331086158752441), (133419, 6.0373430..."
4,540,"[(157296, 6.175747871398926), (1947, 6.1094632..."
...,...,...
605,208,"[(417, 10.020978927612305), (79134, 8.16646671..."
606,401,"[(3067, 5.685152053833008), (1189, 5.473793983..."
607,422,"[(1232, 6.686619281768799), (3355, 5.656732559..."
608,517,"[(3155, 8.061444282531738), (110771, 7.2583098..."


In [214]:
def movies_for_user(user):
    movie_ids = []
    movie_titles = []
    movies = result_recomendation_user[result_recomendation_user["userId"] == user]["recommendations"].tolist()[0]
    for movie in movies:
        movie_ids.append(movie[0])
    for Id in movie_ids:
        movie_titles.append(df_movies[df_movies["movieId"] == Id]["title"])
    return movie_titles

def recommended_movies(users):
    all_user_movies = []
    for user in users:
        all_user_movies.append( (user,list(itertools.chain.from_iterable(movies_for_user(user))) ) )
    return all_user_movies

In [215]:
users = [1,8,2,5]
all_user_movies = recommended_movies(users)

In [216]:
all_user_movies

[(1,
  ['The Lego Movie (2014)',
   'The Artist (2011)',
   'Reign Over Me (2007)',
   'Fantastic Mr. Fox (2009)',
   'Zack and Miri Make a Porno (2008)',
   'Last Detail, The (1973)',
   'Moonrise Kingdom (2012)',
   'Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)',
   'In the Mood For Love (Fa yeung nin wa) (2000)',
   'Sophie Scholl: The Final Days (Sophie Scholl - Die letzten Tage) (2005)',
   'Roman Holiday (1953)',
   'Stalker (1979)',
   'Good Girl, The (2002)',
   'Big Short, The (2015)',
   "Adam's Rib (1949)",
   'Captain Phillips (2013)',
   'Center Stage (2000)',
   'Chorus, The (Choristes, Les) (2004)',
   'Once Were Warriors (1994)',
   'Dick (1999)']),
 (8,
  ['Henry Fool (1997)',
   'High Noon (1952)',
   'Hannah and Her Sisters (1986)',
   "Buffalo '66 (a.k.a. Buffalo 66) (1998)",
   'Topo, El (1970)',
   'Seventh Seal, The (Sjunde inseglet, Det) (1957)',
   'Bridge on the River Kwai, The (1957)',
   'Babe: Pig in the City (1998)',
   'Akira (1988)',
   "Spanish Apartme

# Evaluation

In [123]:
true_relevance = prediction.groupby("userId", sort = False)["rating"].apply(list)
true_relevance = true_relevance.reset_index()

In [124]:
predict_rating = prediction.groupby("userId", sort = False)["prediction"].apply(list)
predict_rating = predict_rating.reset_index()

In [125]:
sampels = len(real_rating) # take number of my query 
ndcg= []
def compute_avg_ndcg(predict_relevance):
    #sort y actual data and remove none value to find ndcg
    for row in range(len(true_relevance["rating"])):
        if(len(true_relevance["rating"][row]) > 1):
            true_relevance["rating"][row].sort(reverse=True)
    
            y1 = np.asarray([true_relevance["rating"][row]])
            y2 = np.asarray([predict_relevance["prediction"][row]])
    
            y1[y1 == None] = 0
            y2[y2 == None] = 0
    
            ndcg.append(ndcg_score(y1, y2))

    #find average ndcg for each query
    avg_NDCG = sum(ndcg)/len(ndcg)
    return avg_NDCG

In [126]:
NDCG_AVG = compute_avg_ndcg(predict_rating)
print(NDCG_AVG) 

0.9415356191336732
